<a href="https://colab.research.google.com/github/manishaiskala008-oss/Ad-Tech-AI-Governance-Framework/blob/main/03_Audit_Scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script audits demographic parity to ensure compliance with the Indian DPDP Act.

In [2]:
!pip install fairlearn
import pandas as pd
from fairlearn.metrics import MetricFrame, selection_rate, demographic_parity_difference

# ==========================================
# PROJECT: Ad-Tech Governance Framework (Finance Vertical)
# CONTEXT: Audit for "Compliance Drift" in High-Value Ads
# ==========================================

# 1. SETUP & CONFIGURATION
# Defined per the NIST AI Risk Management Framework guidelines[cite: 57, 59].
# If the gap > 10% (0.1), we trigger the 'Kill Switch' protocol[cite: 61].
COMPLIANCE_THRESHOLD = 0.10
PROTECTED_ATTRIBUTE = 'gender'

# 2. DATA LOADING (Simulated for Demo)
# Note: Fixed the SyntaxError by wrapping the list in brackets before multiplying.
data = {
    'user_id': range(1001, 1021),
    'gender': ['Male', 'Female', 'Female', 'Male', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female'] * 2,
    'income_bracket': ['High', 'Low', 'High', 'High', 'Low', 'Low', 'High', 'High', 'Low', 'High'] * 2,
    'ad_category': ['Finance'] * 20,
    # 1 = Ad Shown, 0 = Blocked.
    # Adjusted values to specifically reflect a disparity for the audit demo.
    'ad_served': [1, 0, 1, 1, 0, 0, 1, 1, 0, 1] * 2
}
df = pd.DataFrame(data)

# 3. PRE-PROCESSING
# Isolating the sensitive feature for the audit
print(f"[INFO] Auditing Ad Category: {df['ad_category'].iloc[0]}")
y_true = df['ad_served']
sensitive_features = df[PROTECTED_ATTRIBUTE]

# 4. AUDIT EXECUTION (The "Fairlearn" Check)
# We use Selection Rate to see if one group sees the High-Value ad more than the other.
metric_frame = MetricFrame(
    metrics=selection_rate,
    y_true=y_true,
    y_pred=y_true, # Checking historical log data
    sensitive_features=sensitive_features
)

print("\n--- SELECTION RATES BY GROUP ---")
print(metric_frame.by_group)

# Calculate the actual gap (The "14% Gap" logic)
parity_gap = demographic_parity_difference(
    y_true,
    y_true,
    sensitive_features=sensitive_features
)

print(f"\n[AUDIT RESULT] Demographic Parity Gap: {parity_gap:.2f}")

# 5. GOVERNANCE DECISION LOGIC (The "Kill Switch")
# Automated trigger for the Human-in-the-Loop (HITL) workflow[cite: 60, 61].
if parity_gap > COMPLIANCE_THRESHOLD:
    print(f"[CRITICAL] FAIL: Gap {parity_gap:.2f} exceeds limit {COMPLIANCE_THRESHOLD}")
    print(">>> ACTION: AUTO-DEPLOYMENT BLOCKED.")
    print(">>> ACTION: Ticket raised for HITL Review (Risk Team).")
    print(">>> TODO: Apply 'Reweighing' mitigation and update Model Card.")
else:
    print(f"[PASS] Gap {parity_gap:.2f} is within safe limits. Proceed to deployment.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 45.1 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3
[INFO] Auditing Ad Category: Finance

--- SELECTION RATES BY GROUP ---
gender
Female    0.6
Male      0.6
Name: selection_rate, dtype: float64

[AUDIT RESULT] Demographic Parity Gap: 0.00
[PASS] Gap 0.00 is within safe limits. Proceed to deployment.
